# ⾦融计算机语⾔讲义

## 第 8 课 Numba 及其并⾏化

## 学习任务

In [18]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_activities = 'all'

In [19]:
import numpy as np
import quantecon as qe
import matplotlib.pyplot as plt

%matplotlib inline

### 1 阅读
阅读教材 [“The Scientific Libraries: Numba”](https://python-programming.quantecon.org/numba.html) ⼀节，学习有关 numba 软件包 (package) 的基础知识。

### 2 简答题
numpy 软件包提供的向量化 (vectorization) ⽅法有哪两处缺点使得其并不能完全解决数值计算的效率
问题？

#### 答

1. 在处理大量数据时，它需要高度密集型的内存(短时间内消耗大量的内存等计算机资源)

2. 并不是所有的算法集可以完全向量化的（可以完全向量化的算法集并不是普遍存在的）

### 3 简答题
请简要解释 “即时编译” (just in time compilation, JIT) 的核⼼思想。

#### 答
核心思想：动态地将函数编译成本机代码指令，成功时,编译后的代码非常快。

（The key idea is to compile functions to native machine code instructions on the fly.）

个人理解：jit将动态的python代码编译静态化以提高运行速度，也就是会在某种程度上“预编译”输入的代码，这意味着它会在某种程度上固定住各个变量的数据类型

数据量越大、并发的效果越明显。反之，数据量小的时候，并发很有可能会降低性能

### <font color = red>4 简答题
以下代码所定义的 qm 函数 (function)

In [20]:
import numpy as np
α = 4.0
def qm(x0, n):
    x = np.empty(n+1)
    x[0] = x0
    for t in range(n):
        x[t+1] = α * x[t] * (1 - x[t])
    return x
qm(0.1,10)

array([0.1       , 0.36      , 0.9216    , 0.28901376, 0.82193923,
       0.58542054, 0.97081333, 0.11333925, 0.40197385, 0.9615635 ,
       0.14783656])

是向量化 (vectorized) 的吗？从哪⾥可以看出？

1. 不是矢量化的：只调用了np.empty，其他的均为纯python代码，np.empty这个函数不是矢量化函数,且独立性失效

2. 另一种角度：该函数有逐个元素的显式循环，因而不是是矢量化代码

### 5 简答题

如果已经定义` qm `函数 (function)

In [21]:
import numpy as np
α = 4.0
def qm(x0, n):
    x = np.empty(n+1)
    x[0] = x0
    for t in range(n):
        x[t+1] = α * x[t] * (1 - x[t])
    return x

再键⼊什么代码，能够得到其对应的⽀持即时编译 (JIT) 的函数 (function) qm_numba ？

In [22]:
from numba import jit
qm_numba = jit(qm)


### <font color = red>6 简答题
请简要描述即时编译 (JIT) 的⼯作过程，指出其在哪个阶段对函数 (function) 进⾏编译 (compile)，从⽽
会⼤幅提⾼ Python 函数 (function) 的运⾏速度？

#### 根据你自己理解写一下就行了

- 实际调用这个函数时，比如通过执行qm(0.5, 10)， x0和n的类型就变得清晰了(python已经自动识别并调用了，未执行的话由于python的灵活性，变量输入种类十分多变，难以JIT，x0可以是一个NumPy数组或列表，n可以是一个整数或浮点数，等等。
这使得预编译函数变得困难。)
而一旦输入已知，qm中其他变量的类型就可以推断出来。
- 所以Numba和其他JIT编译器的策略是等待到此时，然后编译函数。
这就是为什么它被称为“即时”编译。


### <font color = red>7 简答题（本题待修改，答案不肯定）
假定函数 (function) `qm_numba `是经过 `numba `的 `jit` 修饰器 (decorator) 修饰过 (decorated) 的⽀持
即时编译 (JIT) 的函数 (function)。请问在修饰 (decorate) 时进⾏编译 (compile) 吗？在第⼀次调⽤
(call) `qm_numba(0.5, 10) `时进⾏编译 (compile) 吗？在第⼆次调⽤ (call) `qm_numba(0.9, 20) `时进
⾏编译 (compile) 吗？为什么？

[No Python mode vs Object mode](https://numba.pydata.org/numba-doc/latest/user/performance-tips.html)

#### 答：
- 在修饰时就进行编译
    理想状况下，Numba 可以推断出所有必要的类型信息，就可以生成本地机器代码，而无需调用 Python 运行时的环境

- 在第一次调用时也进行编译，当 Numba 不能推断出所有的类型信息时，一些 Python 对象被赋予通用对象状态，执行回到 Python 运行环境

- 在第二次调用时不进行编译，因为已经识别或者默认了所有类型信息了，已经转化成本地高效代码

In [23]:
from numba import njit

@njit
def qm(x0, n):
    x = np.empty(n+1)
    x[0] = x0
    for t in range(n):
        x[t+1] = 4 * x[t] * (1 - x[t])
    return x
qe.tic()
qm(0.1,10000)
time0=qe.toc()
qe.tic()
qm(0.1,10000)
time1=qe.toc()
qe.tic()
qm(0.1,10000)
time2 =qe.toc()
time1/time0

TOC: Elapsed: 0:00:0.15
TOC: Elapsed: 0:00:0.00
TOC: Elapsed: 0:00:0.00


0.0

### 8 简答题
在以下代码中，

In [24]:
from numba import jit

@jit
def qm(x0, n):
    x = np.empty(n+1)
    x[0] = x0
    for t in range(n):
        x[t+1] = α * x[t] * (1 - x[t])
    return x

第 3 ⾏ @jit 是什么 Python 语法？其等价于运⾏什么 Python 普通代码？

#### 答：
1. 是numba库的jit修饰器语法

2. 等价于qm = jit(qm)

### <font color = red>9 简答题
numba 软件包 (package) 所提供的 jit 与 njit 两个修饰器 (decorator) 相⽐，有什么区别？

- 一个常见的模式是使用@jit 来装饰函数，因为这是 Numba 提供的最灵活的装饰器。 @ jit 基本上包含两种编译模式，首先它将尝试在非 Python 模式下编译修饰函数，如果失败，`它将再次尝试使用对象模式编译函数`。(当这种情况发生时，Numba 只提供很小的速度增益或根本没有。 



- 虽然在对象模式下使用循环提升可以提高性能，但是让函数在非 python 模式下编译才是获得良好性能的关键。 为了使其只是用no python 模式，(我们通常倾向于在发生错误时强制执行错误，因此我们知道有效的编译是失败的)如果编译失败，则使用@njit 或@jit (nopython True)

### 11 简答题
Python 的 Cython 软件包 (package) 主要提供哪⽅⾯的功能？

### 答：

- 与 Numba 一样，Cython 提供了一种生成可以从 Python 中使用的快速编译代码的方法。


与 Numba 的情况一样，关键问题是 Python 是动态类型的。

Numba 通过推断类型解决了这个问题(如果可能的话)。

Cython 的方法不同--程序员直接将类型定义添加到他们的“ Python”代码中。

- 因此，Cython 语言可以被认为是具有类型定义的 Python。

- 除了语言规范，Cython 还是一个语言翻译器，它将 Cython 代码转换为优化的 c 和 c + + 代码。

- Cython还负责构建语言扩展——在编译后的代码和Python之间进行构建接口的包装器代码

虽然 Cython 有一定的优势，但我们通常发现它比 Numba 更慢更笨重。

### 12 简答题
运⾏以下代码会终端 (terminal) 输出什么？

In [25]:
from numba import jit
a = 1
@jit
def add_a(x):
    return a + x
print(add_a(10))

11


继续运⾏以下代码会在终端 (terminal) 输出什么？

In [26]:
a = 2
print(add_a(10))

11


### 13 阅读
阅读教材 “The Scientific Libraries: Parallelization” ⼀节，学习有关并⾏化 (parallelization) 的基础知
识

### <font color = red>14 简答题
多进程 (multiprocessing) 是什么意思？⼀般在什么硬件环境下使⽤多进程 (multiprocessing) 编程？

#### 答：
1. 多进程意味着使用多个处理器并发执行多个进程（一个进程是一个指令链(例如，一个程序)）

2. 硬件环境：多进程可以在具有多个 cpu 的一台机器上进行，也可以在通过网络连接的一组机器上进行。（在后一种情况下，机器的集合通常称为集群cluster）

对于多进程，每个进程都有自己的内存空间，尽管物理内存芯片可能是共享的

### <font color = red>15 简答题
多线程 (multithreading) 是什么意思？与多进程 (multiprocessing) 的不同之处是什么？

#### 答：
1. 多线程处理类似于多进程
    - 多进程意味着使用多个处理器并发执行多个进程（一个进程是一个指令链(例如，一个程序)）

    - 硬件环境：多进程可以在具有多个 cpu 的一台机器上进行，也可以在通过网络连接的一组机器上进行。（在后一种情况下，机器的集合通常称为集群cluster）


2. 它在执行期间，所有线程共享相同的内存空间，而多进程每个进程都有自己的内存空间，尽管物理内存芯片可能是共享的

### <font color = red>16 简答题
Python 因为全局解释器锁 (global interpreter lock, GIL) 的缘故，多线程 (multithreading) 程序并不能
真正并⾏ (parallelized) 执⾏。这⼀限制是否适⽤于 numpy 或 numba 软件包 (package)？为什么

1. 这并不限制像 NumPy 和 Numba 这样的 scientific libraries

2. 原因：从这些库和 jit 编译的代码中导入的函数运行在低级执行环境中， Python 的固有限制不存在于这些底层环境

### <font color = red>17 简答题 (import threading这一项可能有误解，看看就好)
`numpy `是否会在运算中⾃动使⽤多线程 (multithreading)？被 `numba.vectorize`修饰器 (decorator)
修饰创造出来的向量化函数 (ufunc) 是否会⾃动使⽤多线程 (multithreading)？如何开启多线程
(multithreading) 功能？

### 答：
1. 会：numpy 会在运算中⾃动使⽤多线程 

2. 会：numba.vectorize 修饰器修饰创造出来的向量化函数 (ufunc) 会⾃动使⽤多线程

3.   @njit(parallel=True)
    - 使用Numba的高度专门化的JIT编译函数
    - 通过 NumPy 的隐式多线程所获得的并行速度增益
    
    - 通过import threading库


    

#### 下面这个纯属测试

In [36]:
from multiprocessing import Process
import os, time

#计算密集型任务
def work():
    res = 0
    for i in range(100000000):
        res *= i 

if __name__ == "__main__":
    l = []
    print("Ming的电脑有",os.cpu_count(),"核")  # 本机为4核
    start = time.time()
    for i in range(4):
        p = Process(target=work)  # 多进程
        l.append(p)
        p.start()
    for p in l:
        p.join()
    stop = time.time()
    print("计算密集型任务，多进程耗时{stop - start}")

Ming的电脑有 12 核
计算密集型任务，多进程耗时{stop - start}


In [28]:
def f(x, y):
    return np.cos(x**2 + y**2) / (1 + x**2 + y**2)

grid = np.linspace(-3, 3, 5000)
x, y = np.meshgrid(grid, grid)

### <font color = red>2018 简答题 个人电脑会不一样，根据自己的来写，我已经尽可能写的通用化了
以下⽤ numpy 创造出来的向量化函数 (ufunc)

In [29]:
%%time
import numpy as np
def f(x, y):
    return np.cos(x**2 + y**2) / (1 + x**2 + y**2)
f(x, y)

Wall time: 1.07 s


array([[0.03475351, 0.03448109, 0.03420679, ..., 0.03420679, 0.03448109,
        0.03475351],
       [0.03448109, 0.03420668, 0.0339304 , ..., 0.0339304 , 0.03420668,
        0.03448109],
       [0.03420679, 0.0339304 , 0.03365215, ..., 0.03365215, 0.0339304 ,
        0.03420679],
       ...,
       [0.03420679, 0.0339304 , 0.03365215, ..., 0.03365215, 0.0339304 ,
        0.03420679],
       [0.03448109, 0.03420668, 0.0339304 , ..., 0.0339304 , 0.03420668,
        0.03448109],
       [0.03475351, 0.03448109, 0.03420679, ..., 0.03420679, 0.03448109,
        0.03475351]])

与以下⽤ numba 创造出来的向量化函数 (ufunc)

In [30]:
%%time
from numba import vectorize
@vectorize
def f_vec(x, y):
    return np.cos(x**2 + y**2) / (1 + x**2 + y**2)
f_vec(x, y)

Wall time: 222 ms


array([[0.03475351, 0.03448109, 0.03420679, ..., 0.03420679, 0.03448109,
        0.03475351],
       [0.03448109, 0.03420668, 0.0339304 , ..., 0.0339304 , 0.03420668,
        0.03448109],
       [0.03420679, 0.0339304 , 0.03365215, ..., 0.03365215, 0.0339304 ,
        0.03420679],
       ...,
       [0.03420679, 0.0339304 , 0.03365215, ..., 0.03365215, 0.0339304 ,
        0.03420679],
       [0.03448109, 0.03420668, 0.0339304 , ..., 0.0339304 , 0.03420668,
        0.03448109],
       [0.03475351, 0.03448109, 0.03420679, ..., 0.03420679, 0.03448109,
        0.03475351]])

#### 答：
1. numba创造出来的更快(974>215)

2. 原因:
    - Numba 和 NumPy 都使用专门针对这些浮点操作的高效机器代码。但是，NumPy 使用的代码在某些方面效率较低。原因是，在 NumPy 中，运算 np.cos (x * * 2 + y * * 2) / (1 + x * * 2 + y * * 2)生成几个中间数组,Numba 通过编译一个专门针对整个操作的函数来避免创建所有这些中间数组。
    - 如果Numba 代码不能更快，那就是 NumPy 弥补了它在隐式多线程方面的不足，但是我的计算机上运行差距明显（2-4倍），数据越大应当差距越大



### <font color = red>2019 简答题
以下代码经过怎样的修改，就能够发挥出 numba 多线程 (multithreading) 并⾏ (parallel) 执⾏的能⼒

In [31]:
from numpy.random import randn
from numba import njit
@njit
def h(w, r=0.1, s=0.3, v1=0.1, v2=1.0):
    """
    Updates household wealth.
    """
    # Draw shocks
    R = np.exp(v1 * randn()) * (1 + r)
    y = np.exp(v2 * randn())
    # Update wealth
    w = R * s * w + y
    return w
@njit
def compute_long_run_median(w0=1, T=1000, num_reps=50_000):
    obs = np.empty(num_reps)
    for i in range(num_reps):
        w = w0
        for t in range(T):
            w = h(w)
        obs[i] = w
    return np.median(obs)

### 答：
1. 给@njit添加并行的 True 标志，并将def 后的范围range更改为prange:
    
2. 实操

    `- @njit`
    
    `-  for i in range(num_reps):`


---------改为：------------------------------------------------------------


    
     `- @njit(parallel=True)`

     `- for i in prange(num_reps):`
   - 添加`from numba import prange`
   

compute_long_run_median()改为compute_long_run_median_parallel()可改可不改

In [32]:
from numpy.random import randn
from numba import njit
from numba import prange

@njit
def h(w, r=0.1, s=0.3, v1=0.1, v2=1.0):
    """
    Updates household wealth.
    """
    # Draw shocks
    R = np.exp(v1 * randn()) * (1 + r)
    y = np.exp(v2 * randn())
    # Update wealth
    w = R * s * w + y
    return w

@njit(parallel=True)
def compute_long_run_median_parallel(w0=1, T=1000, num_reps=50_000):

    obs = np.empty(num_reps)
    for i in prange(num_reps):
        w = w0
        for t in range(T):
            w = h(w)
        obs[i] = w

    return np.median(obs)
qe.tic()
compute_long_run_median_parallel()
qe.toc()

TOC: Elapsed: 0:00:1.62


1.629378318786621

### <font color = red>20 简答题
⼀般来说，什么样的运算任务适合并⾏化 (parallelization) 处理，什么样的运算任务不适合并⾏化
(parallelization) 处理？

### 答：
- 循环中的各个任务彼此独立的运算任务适合并行化处理
- 循环中的各个任务彼此不独立或者独立性较差的运算任务不适合并行化处理（各个任务都依赖某个步骤或者互相作用比较多）

（各个任务不满足独立性的话并行化通常会失败。）

（例如，内部循环中的每个步骤都依赖于最后一个步骤，因此独立性就失败了）

#这就是为什么我们使用普通范围range而不是标准范围prange）

此外
- 一般而言，当并行化的单个任务相对于总执行时间来说非常小时，并行化的效率就会降低。

- 将所有这些小任务分布到多个 cpu 上反而会升高成本

- 'prange' looks like a Numba internal function, has it been imported (i.e. 'from numba import prange')?

In [33]:
from numba import prange

@njit(parallel=True)
def compute_long_run_median_parallel(w0=1, T=1000, num_reps=50_000):

    obs = np.empty(num_reps)
    for i in prange(num_reps):
        w = w0
        for t in range(T):
            w = h(w)
        obs[i] = w

    return np.median(obs)


In [34]:
%%time
compute_long_run_median_parallel()

Wall time: 1.29 s


1.8413718883752708